In [1]:
import pandas as pd
import json

In [2]:
with open("city.list.json", 'r') as json_file:
    data = json.load(json_file)

In [3]:
df = pd.json_normalize(data)

In [4]:
df.shape

(209579, 6)

In [5]:
df.head()

,id,name,state,country,coord.lon,coord.lat
0,833.0,Ḩeşār-e Sefīd,,IR,47.159401,34.330502
1,2960.0,‘Ayn Ḩalāqīm,,SY,36.321911,34.940079
2,3245.0,Taglag,,IR,44.983330,38.450001
3,3530.0,Qabāghlū,,IR,46.168499,36.173302
4,5174.0,‘Arīqah,,SY,36.483360,32.889809


In [6]:
df = df.drop_duplicates(subset="name")
df['id'] = df['id'].astype('Int32')
df.shape

(157606, 6)

In [8]:
!pip install pycountry

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 1.5 MB/s eta 0:00:0000:010:010m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681833 sha256=3648fee43786e3035617857dcb98d9ae8d4bbfc950cfa0a610ff4849f9f2aa19
  Stored in directory: /Users/mac/Library/Caches/pip/wheels/47/15/92/e6dc85fcb0686c82e1edbcfdf80cfe4808c058813fed0baa8f
Successfully built pycountry

[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [40]:
import pycountry

def get_country_name(country_code):
    try:
        return pycountry.countries.get(alpha_2=country_code).name
    except AttributeError:
        return None
    
df['country_name'] = df['country'].apply(get_country_name)

In [41]:
df.head()

,id,name,state,country,coord.lon,coord.lat,country_name
0,833,Ḩeşār-e Sefīd,,IR,47.159401,34.330502,"Iran, Islamic Republic of"
1,2960,‘Ayn Ḩalāqīm,,SY,36.321911,34.940079,Syrian Arab Republic
2,3245,Taglag,,IR,44.983330,38.450001,"Iran, Islamic Republic of"
3,3530,Qabāghlū,,IR,46.168499,36.173302,"Iran, Islamic Republic of"
4,5174,‘Arīqah,,SY,36.483360,32.889809,Syrian Arab Republic


In [42]:
df[df['country_name'] == None]

,id,name,state,country,coord.lon,coord.lat,country_name


In [43]:
df[df['name'] == "Phnom Penh"]

,id,name,state,country,coord.lon,coord.lat,country_name
43815,1821306,Phnom Penh,,KH,104.916008,11.56245,Cambodia


In [44]:
records = df.to_dict(orient='records')

for record in records:
    coord = {}
    for col, value in record.items():
        if '.' in col:
            parts = col.split('.')
            d = coord
            for part in parts[:-1]:
                d = d.setdefault(part, {})
            d[parts[-1]] = value
    record.update(coord)

filtered_data = [{key: value for key, value in record.items() if key not in ('coord.lon', 'coord.lat', 'state', 'country')} for record in records]

# Write the JSON data to a file
with open('city_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(filtered_data, json_file, ensure_ascii=False, indent=4)


In [45]:
len(filtered_data)

157606